In [84]:
import pandas as pd
import geopandas as gpd
import numpy as np
import time
import gspread
import subprocess
from shapely.geometry import MultiPolygon
import pickle
from all_config import *
import os
import save_file_to_s3 as save_file_to_s3
from shapely import wkt
from datetime import datetime, timedelta 
# import logger activate base env and install logger 
from all_config import geo_mapping

today_date = datetime.today()
# Set up a log file for testing
# log_file_path = os.path.join(logpath, f"testing_log{today_date}.txt")
# logger.basicConfig(filename=log_file_path, level=logger.DEBUG, 
#                    format='%(asctime)s - %(levelname)s - %(message)s')

# logger.info("Logger initialized for testing.")



In [81]:
gogptunits = 'trackers/europe/Global Oil and Gas Plant Tracker (GOGPT) - August 2025 DATA TEAM COPY - Gas & Oil Units.csv'
gogptsub = 'trackers/europe/Global Oil and Gas Plant Tracker (GOGPT) - August 2025 DATA TEAM COPY - sub-threshold units.csv'

In [108]:
dfs = []
for file in [gogptunits, gogptsub]:
    df = pd.read_csv(file)
    dfs.append(df)


gogptdfs = pd.concat(dfs).reset_index(drop=True)


In [109]:
gogptdfs.columns
geocol = 'Country/Area'
fuelcol = 'Fuel'
geo = 'europe'
fuel = 'gas'
mapname = 'europe'


In [110]:
# bring over from main file

def split_countries(country_str):

    for sep in [';', '-', ',']:
        if sep in country_str:
            return country_str.strip().split(sep)
        return [country_str]
    
def check_list(row_list, needed_geo):
    return any(item in needed_geo for item in row_list)


def create_filtered_fuel_df(df, trackeracro): 
    # trackeracro, self.fuelcol
    # if trackeracro == 'GOGET':
    #     drop_row = []
    #     print(f'Length of goget before oil drop: {len(df)}')
    #     for row in df.index:
    #         if df.loc[row, 'Fuel type'] == 'oil':
    #             drop_row.append(row)
            
    #     df.drop(drop_row, inplace=True)        
    #     print(f'Length of goget after oil drop: {len(df)}')
    #     # input('Check the above to see if gas only for goget!')
    
    # elif trackeracro in ['GGIT-eu', 'GGIT', 'EGT-gas']:
    #     drop_row = []
    #     print(f'Length of ggit before oil drop: {len(df)}')
    #     for row in df.index:
    #         if df.loc[row, 'Fuel'] == 'Oil':
    #             drop_row.append(row)
    #         elif df.loc[row, 'Fuel'] == '':
    #             drop_row.append(row)
        
    #     df.drop(drop_row, inplace=True)
    #     print(f'len after gas only filter {trackeracro} {len(df)}')

    
    if trackeracro in ['GOGPT']: # GOGPT-eu # if GOGPT-eu does not need to be run on hydrogen tab, also does not need to be run on 'GOGPT-eu' because it was pre filtered for us
        drop_row = []

        for row in df.index:
            
            fuel_cat_list = df.loc[row, 'Fuel'].split(',')
            new_fuel_cat_list = []
            for fuel in fuel_cat_list:
                fuel = fuel.split(':')[0]
                new_fuel_cat_list.append(fuel)
            
            if len(new_fuel_cat_list) > 1:
                # if all in list is fossil liquids
                if new_fuel_cat_list.count('fossil liquids') == len(new_fuel_cat_list):
                    drop_row.append(row)
                # if just one in there and it is fossil liquids
            elif new_fuel_cat_list == ['fossil liquids']:
                drop_row.append(row)
        
        df.drop(drop_row, inplace=True)
        print(f'len after gas only filter {trackeracro} {len(df)}')
                
    
    return df


def create_filtered_geo_fuel_df(trackeracro, geo, fuel, df, fuelcol, geocol, mapname):
    needed_geo = geo_mapping[geo]
    print(f'length of df: {len(df)}')
    if trackeracro != 'GOGET' and trackeracro != 'GOGPT-eu': 
        # not needed for gogpt eu because PM filtered
        geocollist = geocol.split(';')
        print(f'Getting geo: {geo} from col list: {geocollist} for {trackeracro}')
            
        if geo != ['global'] or geo != ['']:
            if len(geocollist) > 1:
                df.columns = df.columns.str.strip()
                # print(geocollist)
                # input('check what geocol list is')
                print('do multi-column search')
                # print(df)
                df['country_to_check'] = [[] for _ in range(len(df))]
                for row in df.index:
                    for col in geocollist:
                        if col in df.columns:
                            df.at[row, 'country_to_check'] += [df.at[row, col]] # issue
                        # elif col.lower() in df.columns:
                        #     df.at[row, 'country_to_check'] += [df.at[row, col.lower()]] # issue
                        else:
                            print(f'{col} geo col not in df for {mapname}')
                            # print(df.columns)
                            # for col in df.columns:
                            #     print(col)
                            # input('look into it')
                filtered_df = df[df['country_to_check'].apply(lambda x: check_list(x, needed_geo))]
                df = filtered_df

            else:
                if geocol in df.columns:
                    df['country_to_check'] = df[geocol].apply(lambda x: split_countries(x) if isinstance(x, str) else [])
                    print(df['country_to_check'])
                else:
                    print(f"Column '{geocol}' not found in data for {mapname}.")
                    [print(col) for col in df.columns]

                filtered_df = df[df['country_to_check'].apply(lambda x: check_list(x, needed_geo))]
                df = filtered_df

        if fuel != ['none']:
            filtered_df = create_filtered_fuel_df(df, trackeracro)
            df = filtered_df
        
        else:
            # print(len(df))
            # print(f"Length of df for normal case: {len(df)}")
            print(f"Length of df for normal case: {len(df)}")

    
    elif trackeracro == 'GOGET':
        main, prod = df  # Unpack the tuple
        to_merge = []
        # fueldf = None
        for df in [main, prod]:
            df.columns = df.columns.str.strip()
            df['country_to_check'] = df[geocol].apply(lambda x: split_countries(x) if isinstance(x, str) else [])
            
            if geo != ['global'] or geo != ['']:
                print(f'geo not global or empty so filters via needed geo')
                df = df[df['country_to_check'].apply(lambda x: check_list(x, needed_geo))]
            if fuel != ['none']:
                if fuelcol in df.columns:
                    fueldf = create_filtered_fuel_df(df, trackeracro)
                    to_merge.append(fueldf)
                else:
                    to_merge.append(df)
            else:
                to_merge.append(df)
        
        filtered_main = to_merge[0] # if there is a fuel filter this main would already be filtered since its the one with fuelcol
        filtered_prod = to_merge[1]
        if fuel != ['none']:
            # creates list of unit ids only on correct fuel type
            # then filter
            gas_goget_ids = fueldf['Unit ID'].to_list()
            filtered_prod = to_merge[1] # already filtered by fuel because in this case it IS fueldf
            print(f'Yes {fueldf} == {filtered_main}')
            print(len(filtered_main))
            print(len(fueldf))
            # input('check if length the same for fueldf and filtered main in yes fuel option')
            filtered_prod = filtered_prod[filtered_prod['Unit ID'].isin(gas_goget_ids)]
            # main would already be filtered above because it has fuel col so this is filtering prod
            # filtered_main = filtered_main[filtered_main['Unit ID'].isin(gas_goget_ids)]
        
        else:
            print('no fuel filter needed')


        df = (filtered_main, filtered_prod)
    elif trackeracro == 'GOGPT-eu':
        print('Pass for gogpt eu')
    else:
        print('Nothing should be printed here, length of df is 1 or 2 if its goget tuple')
        logger.info('Check create_filtered_geo_df')

    return df



In [111]:
filtered_gogpt = create_filtered_geo_fuel_df('GOGPT', geo, fuel, gogptdfs, fuelcol, geocol, mapname) 

length of df: 14407
Getting geo: europe from col list: ['Country/Area'] for GOGPT
0        [Afghanistan]
1        [Afghanistan]
2        [Afghanistan]
3        [Afghanistan]
4        [Afghanistan]
             ...      
14402          [Yemen]
14403          [Yemen]
14404          [Yemen]
14405          [Yemen]
14406          [Yemen]
Name: country_to_check, Length: 14407, dtype: object
len after gas only filter GOGPT 1920


/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_80081/2125957423.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(drop_row, inplace=True)


In [112]:
filtered_gogpt

,GEM location ID,GEM unit ID,Wiki URL,Country/Area,Plant name,Plant Name in Local Language / Script,Other Name(s),Unit name,Fuel,Capacity (MW),...,State/Province,Subregion,Region,Other IDs (location),Other IDs (unit),Captive industry use,Captive industry type,Captive non-industry use,Fuel classification?,country_to_check
7,L100000408827,G100000413019,https://www.gem.wiki/Roskovec_power_station,Albania,Roskovec power station,NaN,Roskovec CCPP Thermal Power Plant,1,fossil gas: natural gas,170.0,...,Fier,Southern Europe,Europe,Beyond Fossil Fuels: AL-1,Beyond Fossil Fuels: AL-1-1,NaN,NaN,NaN,Gas only,[Albania]
10,L100000406417,G100000406378,https://www.gem.wiki/Vlora_power_station,Albania,Vlora power station,Влёрская тепловая электростанция (ТЭС) (Russian),"TEC Vlora, Vlora TPP",1,fossil gas: LNG,100.0,...,Vlorë,Southern Europe,Europe,Beyond Fossil Fuels: AL-2,Beyond Fossil Fuels: AL-2-1,NaN,NaN,NaN,LNG only,[Albania]
11,L100000406417,G100001006518,https://www.gem.wiki/Vlora_power_station,Albania,Vlora power station,Влёрская тепловая электростанция (ТЭС) (Russian),"TEC Vlora, Vlora TPP",2,fossil gas: LNG,150.0,...,Vlorë,Southern Europe,Europe,Beyond Fossil Fuels: AL-2,NaN,NaN,NaN,NaN,LNG only,[Albania]
12,L100000406417,G100001006520,https://www.gem.wiki/Vlora_power_station,Albania,Vlora power station,Влёрская тепловая электростанция (ТЭС) (Russian),"TEC Vlora, Vlora TPP",3,fossil gas: LNG,100.0,...,Vlorë,Southern Europe,Europe,Beyond Fossil Fuels: AL-2,NaN,NaN,NaN,NaN,LNG only,[Albania]
436,L100000400000,G100000400000,https://www.gem.wiki/Donaustadt_power_station,Austria,Donaustadt power station,NaN,NaN,"Unit 1, timepoint 1",fossil gas: natural gas,395.0,...,Vienna,Western Europe,Europe,"Beyond Fossil Fuels: AT-15, ENTSO-E: 14W-PROD-...","Beyond Fossil Fuels: AT-15-1, ENTSO-E: 14W-GEN...",NaN,NaN,NaN,Gas only,[Austria]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13910,L100000408811,G100000412821,https://www.gem.wiki/Cassino_CHP_power_station,Italy,Cassino CHP power station,NaN,NaN,IC,"fossil gas: natural gas, fossil liquids: diesel",0.0,...,Frosinone,Southern Europe,Europe,Beyond Fossil Fuels: IT-122,Beyond Fossil Fuels: IT-122-1,NaN,NaN,NaN,Multi fuel,[Italy]
13911,L100000400408,G100000400408,https://www.gem.wiki/Nera_Montoro_power_station,Italy,Nera Montoro power station,NaN,NaN,1,fossil gas: natural gas,9.0,...,Terni,Southern Europe,Europe,NaN,NaN,both,chemicals,power,Gas only,[Italy]
14104,L100001000010,G100001000011,https://www.gem.wiki/Zachód_power_station,Poland,Zachód power station,NaN,NaN,IC1-3,fossil gas: natural gas,8.5,...,NaN,Eastern Europe,Europe,NaN,NaN,NaN,NaN,NaN,Gas only,[Poland]
14212,L100000400641,G100000406346,https://www.gem.wiki/Sostanj_power_station,Slovenia,Sostanj power station,NaN,Šoštanj power station,PT 53,fossil gas: natural gas,0.0,...,Šoštanj,Southern Europe,Europe,Beyond Fossil Fuels: SI-3,Beyond Fossil Fuels: SI-3-3,NaN,NaN,NaN,Gas only,[Slovenia]


In [113]:
# europe map debug less projects look into pci5 files
curreu = '../testing-gem-tracker-maps/trackers/europe/compilation_output/europe_i5_2025-05-05.geojson'
mapeu = 'trackers/europe/compilation_output/europe_map_2025-08-19.geojson'

currdfeu = gpd.read_file(curreu)

mapdfeu = gpd.read_file(mapeu)

In [115]:
len(mapdfeu[mapdfeu['tracker-acro']=='GOGPT'])

1920

In [116]:
missingfromorig = set(filtered_gogpt['GEM unit ID']) - set(mapdfeu['id'])

In [117]:
missingfromorig

set()

In [118]:
# print(currdfeu.columns) # id
# print(mapdfeu.columns) # id

missingid = set(currdfeu['id'].to_list()) - set(mapdfeu['id'].to_list())


In [28]:
len(missingid) #  220 

220

In [19]:
missingid

{'G100000400703',
 'G100000400704',
 'G100000406309',
 'G100000406348',
 'G100000407324',
 'G100000409959',
 'G100000409978',
 'G100000409979',
 'G100000410143',
 'G100000410144',
 'G100000412709',
 'G100000412713',
 'G100000412714',
 'G100000412715',
 'G100000412716',
 'G100000412717',
 'G100000412718',
 'G100000412720',
 'G100000412721',
 'G100000412722',
 'G100000412723',
 'G100000412724',
 'G100000412725',
 'G100000412726',
 'G100000412727',
 'G100000412729',
 'G100000412730',
 'G100000412731',
 'G100000412732',
 'G100000412734',
 'G100000412735',
 'G100000412736',
 'G100000412737',
 'G100000412738',
 'G100000412739',
 'G100000412751',
 'G100000412752',
 'G100000412753',
 'G100000412775',
 'G100000412776',
 'G100000412777',
 'G100000412778',
 'G100000412793',
 'G100000412794',
 'G100000412795',
 'G100000412796',
 'G100000412798',
 'G100000412799',
 'G100000412800',
 'G100000412801',
 'G100000412802',
 'G100000412815',
 'G100000412816',
 'G100000412827',
 'G100000412828',
 'G1000004

In [ ]:
orig_egt_gas_pipe = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/EGT-gas/2025-02/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02%20DATA%20TEAM%20COPY.geojson'
egtpipedf = gpd.read_file(orig_egt_gas_pipe)
// idsorig_egt_gas_pipe = egtpipedf[]

In [ ]:

# show orig if id is in missing set
currmap_missing = currdfeu[currdfeu['id'].isin(missingid)]
currmap_missing

In [122]:
set(currmap_missing['areas'].to_list())

{'Albania;',
 'Albania; Italy;',
 'Albania; Kosovo;',
 'Albania; Montenegro; Croatia;',
 'Algeria; Italy;',
 'Algeria; Morocco; Spain; Portugal;',
 'Algeria; Spain;',
 'Austria;',
 'Austria; Czech Republic;',
 'Austria; Hungary;',
 'Austria; Slovenia;',
 'Austria; Switzerland;',
 'Azerbaijan; Georgia;',
 'Azerbaijan; Georgia; Türkiye;',
 'Belarus;',
 'Belarus; Lithuania;',
 'Belarus; Lithuania; Russia;',
 'Belarus; Poland;',
 'Belarus; Poland; Slovakia;',
 'Belarus; Ukraine;',
 'Belgium;',
 'Belgium; France;',
 'Belgium; France; Luxembourg;',
 'Belgium; Luxembourg;',
 'Belgium; Netherlands;',
 'Bosnia and Herzegovina;',
 'Bosnia and Herzegovina; Croatia;',
 'Bulgaria;',
 'Bulgaria; Greece;',
 'Bulgaria; Serbia;',
 'Bulgaria; Türkiye;',
 'Croatia;',
 'Croatia; Bosnia and Herzegovina;',
 'Croatia; Serbia;',
 'Croatia; Slovenia;',
 'Croatia; Slovenia; Austria;',
 'Cyprus;',
 'Cyprus; Egypt;',
 'Czech Republic;',
 'Czech Republic; Germany;',
 'Czech Republic; Poland;',
 'Denmark;',
 'Denma

In [41]:
len(currmap_missing)

1342

In [123]:
set(currmap_missing['tracker-custom'].to_list())

{'GGIT', 'GGIT-export', 'GGIT-import', 'GOGPT'}

In [44]:
set(currmap_missing['tracker-custom'].to_list()) # set(currmap_missing['tracker-custom'].to_list())

GOGPTcurrmap_missing = currmap_missing[currmap_missing['tracker-custom'] =='GOGPT']
notGOGPTcurrmap_missing = currmap_missing[currmap_missing['tracker-custom'] !='GOGPT']

In [47]:
# GOGPTcurrmap_missing[['fuel-filter', 'status-legend', 'maturity', 'pci-list']]
notGOGPTcurrmap_missing[['areas','name','tracker-custom','fuel-filter', 'status-legend', 'maturity', 'pci-list']]

,areas,name,tracker-custom,fuel-filter,status-legend,maturity,pci-list
0,Estonia; Russia;,Leningrad-Vyborg-State Border Gas Pipeline,GGIT,methane,operating,none,none
1,Russia;,Gryazovets-Vyborg Gas Pipeline,GGIT,methane,operating,none,none
2,Russia; Ukraine;,Novopskov-Aksai-Mozdok Pipeline,GGIT,methane,operating,none,none
3,Ukraine; Russia;,Stavropol-Moscow Gas Pipeline,GGIT,methane,retired-plus,none,none
4,Azerbaijan; Georgia;,Azerbaijan-Georgia-Romania Interconnector Gas ...,GGIT,methane,proposed-plus,none,none
...,...,...,...,...,...,...,...
3301,Belarus;,Torzhok-Minsk-Ivatsevichy Gas Pipeline,GGIT,methane,operating,n,none
3302,Belarus;,Torzhok-Minsk-Ivatsevichy Gas Pipeline,GGIT,methane,operating,n,none
3303,Belarus;,Torzhok-Minsk-Ivatsevichy Gas Pipeline,GGIT,methane,operating,n,none
3304,Belarus;,Torzhok-Minsk-Ivatsevichy Gas Pipeline,GGIT,methane,operating,n,none


In [11]:
mapdfeu_gogpt = mapdfeu[mapdfeu['tracker-custom']=='GOGPT'][['fuel-filter', 'status-legend', 'maturity', 'pci-list']]

In [12]:
mapdfeu_gogpt

,fuel-filter,status-legend,maturity,pci-list
0,,operating,,none
1,,pre-construction-plus,,none
2,,operating,,none
3,,shelved,,none
4,,pre-construction-plus,,none
...,...,...,...,...
4405,,operating,,none
4406,,operating,,none
4407,,operating,,none
4408,,cancelled,,none


In [63]:
eugogpt_units_to_exclude = [
    "G100001031177",
    "G100001031183",
    "G100001031280",
    "G100001031227",
    "G100001031206",
    "G100001030498",
    "G100001030500",
    "G100001030493",
    "G100001030497",
    "G100001018069",
    "G100001031279",
    "G100001030503",
    "G100001030502",
    "G100001018095",
    "G100001030499",
    "G100001018066",
    "G100001030485",
    "G100001030663",
    "G100001030512",
    "G100001030595",
    "G100001030598",
    "G100001030480",
    "G100001030482",
    "G100001030600",
    "G100001030664",
    "G100001030665",
    "G100001030599",
    "G100001030596",
    "G100001030501",
    "G100001030669",
    "G100001030670",
    "G100001030668",
    "G100001030495",
    "G100001030504",
    "G100001031202",
    "G100001031199",
    "G100001046556",
    "G100001046557",
    "G100001020045",
    "G100001031200",
    "G100001048412",
    "G100001048413",
    "G100001048414",
    "G100001048415",
    "G100001048416",
    "G100001030368",
    "G100001030347",
    "G100001030348",
    "G100001030367",
    "G100001030366",
    "G100001030944",
    "G100001030945",
    "G100001030349",
    "G100001030337",
    "G100001030338",
    "G100001030339",
    "G100001030341",
    "G100001031271",
    "G100001031272",
    "G100001030874",
    "G100001030875",
    "G100001030876",
    "G100001031023",
    "G100001031024",
    "G100001031030",
    "G100001031273",
    "G100001030470",
    "G100001030468",
    "G100001030469",
    "G100000413132",
    "G100001030355",
    "G100001030354",
    "G100001030350",
    "G100001030344",
    "G100001030345",
    "G100001030361",
    "G100001048124",
    "G100001048125",
    "G100001028084",
    "G100001028085",
    "G100001030351",
    "G100001030352",
    "G100001048119",
    "G100001030353",
    "G100001030365",
    "G100001048126",
    "G100001048127",
    "G100001048128",
    "G100001030362",
    "G100001030363",
    "G100001048121",
    "G100001028011",
    "G100001028012",
    "G100001055463",
    "G100001055464",
    "G100001055462",
    "G100001031133",
    "G100001031138",
    "G100001031139",
    "G100001031141"
]

In [66]:
set(missingid)

{'G100000400703',
 'G100000400704',
 'G100000406309',
 'G100000406348',
 'G100000407324',
 'G100000409959',
 'G100000409978',
 'G100000409979',
 'G100000410143',
 'G100000410144',
 'G100000412709',
 'G100000412713',
 'G100000412714',
 'G100000412715',
 'G100000412716',
 'G100000412717',
 'G100000412718',
 'G100000412720',
 'G100000412721',
 'G100000412722',
 'G100000412723',
 'G100000412724',
 'G100000412725',
 'G100000412726',
 'G100000412727',
 'G100000412729',
 'G100000412730',
 'G100000412731',
 'G100000412732',
 'G100000412734',
 'G100000412735',
 'G100000412736',
 'G100000412737',
 'G100000412738',
 'G100000412739',
 'G100000412751',
 'G100000412752',
 'G100000412753',
 'G100000412775',
 'G100000412776',
 'G100000412777',
 'G100000412778',
 'G100000412793',
 'G100000412794',
 'G100000412795',
 'G100000412796',
 'G100000412798',
 'G100000412799',
 'G100000412800',
 'G100000412801',
 'G100000412802',
 'G100000412815',
 'G100000412816',
 'G100000412827',
 'G100000412828',
 'G1000004

In [67]:
set(eugogpt_units_to_exclude)

{'G100000413132',
 'G100001018066',
 'G100001018069',
 'G100001018095',
 'G100001020045',
 'G100001028011',
 'G100001028012',
 'G100001028084',
 'G100001028085',
 'G100001030337',
 'G100001030338',
 'G100001030339',
 'G100001030341',
 'G100001030344',
 'G100001030345',
 'G100001030347',
 'G100001030348',
 'G100001030349',
 'G100001030350',
 'G100001030351',
 'G100001030352',
 'G100001030353',
 'G100001030354',
 'G100001030355',
 'G100001030361',
 'G100001030362',
 'G100001030363',
 'G100001030365',
 'G100001030366',
 'G100001030367',
 'G100001030368',
 'G100001030468',
 'G100001030469',
 'G100001030470',
 'G100001030480',
 'G100001030482',
 'G100001030485',
 'G100001030493',
 'G100001030495',
 'G100001030497',
 'G100001030498',
 'G100001030499',
 'G100001030500',
 'G100001030501',
 'G100001030502',
 'G100001030503',
 'G100001030504',
 'G100001030512',
 'G100001030595',
 'G100001030596',
 'G100001030598',
 'G100001030599',
 'G100001030600',
 'G100001030663',
 'G100001030664',
 'G1000010

In [64]:
overlap_missing_h2removed = set(missingid) - set(eugogpt_units_to_exclude)

In [65]:
len(overlap_missing_h2removed)

220

In [9]:
# latam map test no change

# gogpt and coal filter south america list geo compare original global to map

# gogpt orig
# trackerdf_for_GOGPT_on_2025-08-14.pkl

with open(f'local_pkl/trackerdf_for_GOGPT_on_2025-08-14.pkl', 'rb') as f:
    gogptorigdf = pickle.load(f)

# gcpt orig
# trackerdf_for_GCPT_on_2025-08-14.pkl

with open(f'local_pkl/trackerdf_for_GCPT_on_2025-08-14.pkl', 'rb') as f:
    gcptorigdf = pickle.load(f)

In [17]:
# total internal number
internalmap = 'trackers/internal/compilation_output/internal_map_2025-08-18.geojson'
internalmapdf = gpd.read_file(internalmap)



DataSourceError: trackers/internal/compilation_output/internal_map_2025-08-18.geojson: No such file or directory

In [22]:
len(internalmapdf)
print(internalmapdf.columns)
print(set(internalmapdf['tracker-acro'].to_list()))

Index(['pid', 'id', 'url', 'areas', 'name', 'unit-name', 'fuel', 'capacity',
       'status', 'start-year', 'owner', 'parent', 'Latitude', 'Longitude',
       'subnat', 'region', 'tracker-custom', 'tracker-acro', 'noneng-name',
       'retired-year', 'Operator', 'area2', 'binational', 'capacity2',
       'loc-owner', 'operator', 'loc-oper', 'tech-type', 'river', 'loc-accu',
       'subnat2', 'region2', 'Fuel', 'unit-id', 'prod-oil', 'prod-year-oil',
       'prod-gas', 'prod-year-gas', 'owners-noneng', 'prod-coal', 'prod-year',
       'mine-type', 'workforce', 'coal-grade', 'coalfield', 'scaling-capacity',
       'capacity-table', 'units-of-m', 'status-legend',
       'areas-subnat-sat-display', 'name-search', 'owner-search',
       'parent-search', 'capacity-details', 'tracker-display', 'geometry'],
      dtype='object')
{'GGIT', 'GSPT', 'GCPT', 'GNPT', 'GOGPT', 'GBPT', 'GCTT', 'GOGET', 'GGIT-lng', 'GOIT', 'GHPT', 'GGPT', 'GWPT', 'GCMT'}


In [5]:
original_949_giomt = 'testingcode/files/Global Iron Ore Mines Tracker - August 2025 - Standard Copy (V1) DATA TEAM COPY - Main Data.csv'
map_929_giomt = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/giomt/2025-08/giomt_map_2025-08-15.geojson'

orig = pd.read_csv(original_949_giomt)
map = gpd.read_file(map_929_giomt)

In [12]:
print(len(orig)) # GEM Asset ID
print(len(map))  # pid 

set_orig_id = set(orig['GEM Asset ID'].to_list())
set_map_id = set(map['pid'].to_list())

missing = set_orig_id - set_map_id
missing

# show orig if id is in missing set
orig_missing = orig[orig['GEM Asset ID'].isin(missing)]


949
929


In [15]:
missing

{'P100000128376',
 'P100000128562',
 'P100000128563',
 'P100000128564',
 'P100000128565',
 'P100000128566',
 'P100000128620',
 'P100000128646',
 'P100000128695',
 'P100000128699',
 'P100000128798',
 'P100000128822',
 'P100000128823',
 'P100000128830',
 'P100000128851',
 'P100000128855',
 'P100000129062',
 'P100000129063',
 'P100000129340',
 'P100000129341'}

In [13]:
orig_missing

,GEM Asset ID,Asset name (English),Asset name (other language),Coordinates,Coordinate accuracy,Municipality,Subnational unit,Country/Area,Region,Production 2024 (ttpa),...,"Total resource (inferred, indicated and measured, thousand metric tonnes)",Operating status,Start date,Stop date,Owner,Owner GEM Entity ID,Owner name in local language/script,Parent,Parent GEM Entity ID,GEM wiki page URL
89,P100000129062,SIMEC Iron Baron (Middleback Hub) Mine,NaN,"-32.995719,137.157008",exact,Iron Baron,South Australia,Australia,Asia Pacific,unknown,...,"166,667",operating,unknown,NaN,SIMEC Mining [100%],E100002002844 [100.0%],NaN,GFG Alliance Ltd,E100001000556,https://www.gem.wiki/SIMEC_Iron_Baron_(Middleb...
90,P100000129063,SIMEC South Middleback (Middleback Hub) Mine,NaN,"-33.269204,137.115032",exact,Middleback Range,South Australia,Australia,Asia Pacific,unknown,...,"166,667",operating,1986,NaN,SIMEC Mining [100%],E100002002844 [100.0%],NaN,GFG Alliance Ltd,E100001000556,https://www.gem.wiki/SIMEC_South_Middleback_(M...
154,P100000128376,AVG Esperança Mine,Mina Esperança (Portuguese),"-20.110936,-44.225151",exact,Brumadinho,Minas Gerais,Brazil,Central & South America,unknown,...,unknown,operating,unknown,NaN,Grupo AVG [100%],E100002004172 [100.0%],NaN,--,--,https://www.gem.wiki/AVG_Esperança_Mine
163,P100000128830,CSN Casa de Pedra Mine,Mina Casa de Pedra (Portuguese),"-20.462721,-43.923006",exact,Congonhas,Minas Gerais,Brazil,Central & South America,unknown,...,"6,002,000",operating,1913,NaN,CSN Mineração SA [100%],E100002004100 [100.0%],NaN,CSN Mineração SA [100.0%],E100002004100 [100.0%],https://www.gem.wiki/CSN_Casa_de_Pedra_Mine
171,P100000128855,MMI Ipê Mine,Mina Ipê (Portuguese),"-20.116098,-44.308706",exact,Brumadinho,Minas Gerais,Brazil,Central & South America,unknown,...,unknown,operating,2017,NaN,Mineração Morro do Ipe SA [100%],E100002004208 [100.0%],NaN,Trafigura Group Pte Ltd [50.0%]; Mubadala Inve...,E100001014301 [50.0%]; E100000001086,https://www.gem.wiki/MMI_Ipê_Mine
408,P100000129340,Ansteel Dagushan Mine,大孤山铁矿 (Chinese),"41.055176,123.065045",exact,Anshan,Liaoning,China,Asia Pacific,"4,600",...,"340,000",operating,1906,NaN,Ansteel Group Mining Co Dagushan Branch [100%],E100002003888 [100.0%],鞍钢集团矿业有限公司大孤山分公司,--,--,https://www.gem.wiki/Ansteel_Dagushan_Mine
409,P100000129341,Ansteel Yanqianshan Mine,眼前山铁矿 (Chinese),"41.074821,123.154977",exact,Anshan,Liaoning,China,Asia Pacific,"4,680",...,"974,000",operating,1960,NaN,Angang Group Mining Co. Yanqianshan Branch [100%],E100002003890 [100.0%],鞍钢集团矿业有限公司眼前山分公司,--,--,https://www.gem.wiki/Ansteel_Yanqianshan_Mine
681,P100000128562,IMPASCO Jalal Abad Mine,(Persian) معدن جلال آباد IMPASCO,"31.012703,56.425347",exact,Rashk-e Vosta,Kerman,Iran,Middle East,unknown,...,"200,000",operating,unknown,NaN,Iranian Minerals Production and Supply Co. (IM...,E100002003874 [100.0%],شرکت تولید و تامین مواد معدنی ایران [%100] (Pe...,Iranian Mines & Mining Industries Development ...,E100000000547,https://www.gem.wiki/IMPASCO_Jalal_Abad_Mine
682,P100000128563,CMIC Chadormalu Mine,(Persian) معدن چادرملو CMIC,"32.297578,55.522161",exact,Ardakan,Yazd,Iran,Middle East,unknown,...,"400,000",operating,1999-04-01,NaN,Chadormalu Mining and Industrial Company [100%],E100001012537 [100.0%],شرکت معدنی و صنعتی چادرملو [%100] (Persian),Chadormalu Mining & Industrial Co [100.0%],E100001012537 [100.0%],https://www.gem.wiki/CMIC_Chadormalu_Mine
683,P100000128564,Sabanour Gulali Mine,(Persian) معدن گلالی صبانور,"34.991567,47.911152",exact,Qorveh,Kurdistan,Iran,Middle East,unknown,...,"10,000",operating,1993,NaN,Sabanour Mining and Industrial Development Co ...,E100001016086 [100.0%],شرکت توسعه معدنی و صنعتی صبانور [%100] (Persian),--,--,https://www.gem.wiki/Sabanour_Gulali_Mine


Steps to testing map files:
1.) Gather files (source file of tracker data, test map file, test map data download if regional)
2.) Adjust key variables for the test (tracker, newdata, mapname)
3.) Using df.info() or other find key cols to rename source file with (colmapping function)
4.) Run topline stat function 

In [6]:
tfile = "/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/coal-plant/compilation_output/gcpt_map_2025-07-23.geojson"

tdf = gpd.read_file(tfile)
print(tdf.info())
for col in tdf.columns:
    print(f'{col}:\n')
    uniquethings = set(tdf[col].to_list())
    if len(uniquethings) < 30:
        print(uniquethings)
    else:
        print('too unique')
    

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14356 entries, 0 to 14355
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   id                        14356 non-null  object  
 1   pid                       14356 non-null  object  
 2   areas                     14356 non-null  object  
 3   url                       14356 non-null  object  
 4   name                      14356 non-null  object  
 5   unit-name                 14356 non-null  object  
 6   noneng-name               14356 non-null  object  
 7   owner                     14356 non-null  object  
 8   parent                    14356 non-null  object  
 9   capacity                  14356 non-null  object  
 10  status                    14356 non-null  object  
 11  start-year                14356 non-null  object  
 12  retired-year              14356 non-null  object  
 13  subnat                    14356 non-nu

In [2]:
# testing_final_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/final/'

# for file in os.listdir(testing_final_path): 
#     if (map_to_test in file) & file.endswith(".geojson"):
#         final_map = gpd.read_file(testing_final_path + file)
#         print(file)

#     elif (map_to_test in file) & file.endswith(".xlsx") & (new_release_date in file) & ('about' in file):
#         # final_data = pd.ExcelFile(testing_final_path + file)
#         # Read all sheets into a dictionary
#         final_data_dict_with_about = pd.read_excel(testing_final_path + file, sheet_name=None)
#         print(file)
 
#     elif (map_to_test in file) & file.endswith(".xlsx") & (new_release_date in file):
#         # final_data = pd.ExcelFile(testing_final_path + file)
#         # Read all sheets into a dictionary
#         final_data_dict = pd.read_excel(testing_final_path + file, sheet_name=None) 
#         print(file)

# print(f"We will be testing for map {map_to_test}.")
# print(f"Make sure all local source files have been updated for this cycle.")


In [ ]:
# tracker = 'gcmt'
# new_release_date = 'May_2025' # for within about page NEEDS TO BE FULL MONTH
# releaseiso = '2025-05'
# iso_today_date = '2025-05-05'
# GCMT TESTING MAY 2025
# find 5 missing between source dd and test map 
## GLOBAL 
# tfile_map = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/final/{tracker}_map_{iso_today_date}.geojson' # gcmt_map_2025-05-06.geojson
# tfile_dd = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/{tracker}/testing/{tracker}-data-download_{new_release_date}_{iso_today_date}_test.xlsx'

# source_dd = f'/Users/gem-tah/Downloads/GlobalCoalMineTracker,May2025DTC_md_2025-05-05.parquet'
# source_dd = f'/Users/gem-tah/Downloads/Global Coal Mine Tracker, May 2025 DATA TEAM COPY.xlsx'
# last_map = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/coal-mine/compilation_output/data.csv'
source_dd = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/latest/Global-Oil-and-Gas-Extraction-Tracker-Feb-2025_DATA_TEAM_COPY_md_2025-04-16.parquet'
last_map = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/goget/GOGET_Earthgenome_file2024-04-01.csv'

## REGIONAL ## TODO swap out test file paths manually from print out above
# latam = f'{testfolder}latam_map_2025-05-06.geojson'
# africa = f'{testfolder}africa_map_2025-05-06.geojson'
europe = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/mapfiles/europemap2025-03.parquet'
asia = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/mapfiles/asiamap2025-03.parquet'


In [ ]:
# GCMT

# tfile_map_gdf = gpd.read_file(tfile_map)
# latam map file
# latamdf = gpd.read_file(latam)
# africadf = gpd.read_file(africa)

# Create a DataFrame from the CSV last map file
# last_map_df = pd.read_csv(last_map, encoding='utf-16')

# Create a DataFrame from the Excel data download file
# Read all sheets from the Excel file into a dictionary of DataFrames
# tfile_dd_sheets = pd.read_excel(tfile_dd, sheet_name=None)
# tfile_dd_df = pd.concat(tfile_dd_sheets.values(), ignore_index=True)

# source_dd_sheets = pd.read_excel(source_dd, sheet_name=None)
# source_dd_df = pd.concat(source_dd_sheets.values(), ignore_index=True)

# Display the first few rows of each DataFrame to verify
# print(tfile_map_gdf.info()) # 6987 pid, areas, prod-coal, status, url
# # print(last_map_df.info()) # 6580 id, country, production, status, url
# # print(tfile_dd_df.info()) # 
# print(source_dd_df.info()) # 7095 GEM Mine ID, Country / Area, Production (Mtpa), Status,GEM Wiki Page (ENG), GEM Wiki Page (Non-ENG), 



In [22]:
# GOGET
sourcedf = pd.read_parquet(source_dd) 
# cols source: Unit ID Unit Name Country/Area Latitude Longitude Wiki URL
eudf = pd.read_parquet(europe)
# cols map: pid name areas Latitude Longitude url
asiadf = pd.read_parquet(asia)

# print(eudf.info())
set(asiadf['tracker-acro'].to_list())

{'GGIT', 'GGIT-lng', 'GOGET', 'GOGPT'}

In [11]:
def topline_stat(df, map_df, filter='global'):
    print(f'Source Data {newdata} for {filter}: {len(df)}')
    print(f'Map Data {newdata} for {filter}: {len(map_df)}')
    df_missing = df[~df['pid'].isin(map_df['pid'])]
    df_missing_reverse = map_df[~map_df['pid'].isin(df['pid'])]
    
    print('Map Discrepancy Count')
    print(len(df_missing))
    print(df_missing_reverse)
    if len(df_missing) > 0:
        # print(df_missing) 
    
        # add a filter to save missing because of empty geometry but not empty area
        # look at what is left
        # TODO LEFT HERE may 6th 5:37pm
        emptygeodf = df_missing[df_missing['Latitude'].isna()]
        emptygeodf = emptygeodf.dropna()
        # print(emptygeodf)        
        nonemptygeodf = df_missing[~df_missing['Latitude'].isna()]
        for row in nonemptygeodf.index:
            print(nonemptygeodf.loc[row].to_frame().T.to_string(index=False))

In [33]:

newdata = 'GGIT' # EGT-gas, EGT-term,plants, plants_hy,GBPT, GCMT, GCPT, GCTT, GGIT, GGIT-lng, GGPT, GHPT, GNPT, GOGET,GOGPT, GOIT, GSPT, GWPT,
mapname = 'asia'


In [34]:
# most interested in the tracker of interest's total number in regional and global map and dd
# GCMT 219 africa, 52 latam, 6991 global but minus 75 no coords
# write results to topline tab in 15l2fcUBADkNVHw-Gld_kk7EaMiFFi8ysWt6aXVW26n8
# tracker name == official release tab name
# map geo == global, africa, latam, europe, asia
# check that only updated data is diff for regional
# 

# in map class
# if self.name == 'africa':
if mapname == 'africa':
    needed_geo = africa_countries
    
elif mapname == 'asia':
    needed_geo = asia_countries
    
elif mapname == 'latam':
    needed_geo = latam_countries
    
elif mapname == 'europe':
    needed_geo = europe_countries
    
#### TODO first rename
# create a dictionary that maps each files key columns to the mapcols so we can compare easily
mapcols = ['pid', 'areas', 'name', 'status'] 
sourcefile_rename = ['Unit ID', 'Country/Area', 'Unit Name','Status'] # GOGET 2025
# oldmap_rename = ['id', 'country', 'production', 'status']  # id, country, production, status, url
# sourcefile_rename_GCMT = ['GEM Mine ID', 'Country / Area', 'Production (Mtpa)', 'Status']  # # GCMT 2025 GEM Mine ID, Country / Area, Production (Mtpa), Status, GEM Wiki Page (ENG), GEM Wiki Page (Non-ENG)
# ddfile_rename = {} # this would be same as sourcefile rename 

def colmapping(key_list,value_list):
    
    column_mapping = dict(zip(key_list, value_list))
    
    return column_mapping

df = sourcedf.rename(columns=colmapping(sourcefile_rename, mapcols))
df = df[['pid', 'areas', 'status', 'Latitude', 'Longitude', 'Fuel type', 'Fuel']]
map_df = eudf[['pid', 'areas', 'name', 'status', 'geometry', 'tracker-acro']]

map_df['pid'].fillna('',inplace=True)

if newdata == 'GOGPT':
    if mapname == 'europe':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # GEO filter for source data
        
        drop_row = []       # Fuel filter for source data
        for row in df.index:
            fuel_cat_list = df.loc[row, 'Fuel'].split(',')
            new_fuel_cat_list = []
            for fuel in fuel_cat_list:
                fuel = fuel.split(':')[0]
                new_fuel_cat_list.append(fuel)
            
            if len(new_fuel_cat_list) > 1:
                # if all in list is fossil liquids
                if new_fuel_cat_list.count('fossil liquids') == len(new_fuel_cat_list):
                    drop_row.append(row)
                # if just one in there and it is fossil liquids
            elif new_fuel_cat_list == ['fossil liquids']:
                drop_row.append(row)
                
        df.drop(drop_row, inplace=True) 
        
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GOGPT')] # GOGPT ONLY from pre filtered map on geo and fuel
        topline_stat(df, map_df, 'europe')      
           
    elif mapname == 'asia':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            fuel_cat_list = df.loc[row, 'Fuel'].split(',')
            new_fuel_cat_list = []
            for fuel in fuel_cat_list:
                fuel = fuel.split(':')[0]
                new_fuel_cat_list.append(fuel)
            
            if len(new_fuel_cat_list) > 1:
                # if all in list is fossil liquids
                if new_fuel_cat_list.count('fossil liquids') == len(new_fuel_cat_list):
                    drop_row.append(row)
                # if just one in there and it is fossil liquids
            elif new_fuel_cat_list == ['fossil liquids']:
                drop_row.append(row)
                
        df.drop(drop_row, inplace=True) 
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GOGPT')] # GOGPT ONLY
        topline_stat(df, map_df, 'asia')    

elif newdata == 'GGIT-lng':
    if mapname == 'europe':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            if df.loc[row, 'Fuel'] == 'oil':
                drop_row.append(row)
            elif df.loc[row,'Fuel'] == '':
                drop_row.append(row)
                
        df.drop(drop_row, inplace=True) 
        
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GGIT-lng')] # GGIT-lng ONLY
        topline_stat(df, map_df, 'europe')      
           
    elif mapname == 'asia':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            if df.loc[row, 'Fuel'] == 'oil':
                drop_row.append(row)
            elif df.loc[row,'Fuel'] == '':
                drop_row.append(row)
                
        df.drop(drop_row, inplace=True) 
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GGIT-lng')] # GGIT-lng ONLY
        topline_stat(df, map_df, 'asia')        
elif newdata == 'GGIT':
    
    if mapname == 'europe':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            if df.loc[row, 'Fuel'] == 'oil':
                drop_row.append(row)
            elif df.loc[row,'Fuel'] == '':
                drop_row.append(row)
                
        df.drop(drop_row, inplace=True) 
        
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GGIT')] # GGIT ONLY
        topline_stat(df, map_df, 'europe')      
           
    elif mapname == 'asia':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            if df.loc[row, 'Fuel'] == 'oil':
                drop_row.append(row)
            elif df.loc[row,'Fuel'] == '':
                drop_row.append(row)
                
        df.drop(drop_row, inplace=True) 
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GGIT')] # GGIT ONLY
        topline_stat(df, map_df, 'asia')        
            

elif newdata == 'GOGET':
    
    if mapname == 'europe':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            if df.loc[row, 'Fuel type'] == 'oil':
                drop_row.append(row)
        df.drop(drop_row, inplace=True) 
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GOGET')] # GOGET ONLY
        topline_stat(df, map_df, 'europe')      
           
    elif mapname == 'asia':
         #     # sum of GOGET units should be at least 
        df = df[df['areas'].isin(needed_geo)] # asia only
        
        drop_row = []       # gas only  
        for row in df.index:
            if df.loc[row, 'Fuel type'] == 'oil':
                drop_row.append(row)
        df.drop(drop_row, inplace=True) 
        
        map_df = map_df[map_df['tracker-acro'].str.startswith('GOGET')] # GOGET ONLY
        topline_stat(df, map_df, 'asia')        
        

elif newdata == 'GCMT':
    if mapname == 'africa':
         #     # sum of GCMT units should be at least 219
        df = df[df['areas'].isin(needed_geo)] # latam only
        map_df = map_df[map_df['pid'].str.startswith('M')] #GCMT only
        topline_stat(df, map_df, 'africa')
        
    elif mapname == 'latam':
        #     # sum of GCMT units should be at least 52

        df = df[df['areas'].isin(needed_geo)] # latam only
        map_df = map_df[map_df['pid'].str.startswith('M')] #GCMT only
        topline_stat(df, map_df, 'latam')
    
    elif mapname == 'gcmt': # global 
        # sum of GCMT units should be at least 6987 
        print(f'Source Data {newdata}: {len(df)}')
        print(f'Map Data {tracker}: {len(map_df)}')
        df_missing = df[~df['pid'].isin(map_df['pid'])]
        df_missing_reverse = map_df[~map_df['pid'].isin(df['pid'])]
        
        print('Map Discrepancy Count')
        print(len(df_missing))
        print(df_missing_reverse)
        if len(df_missing) > 0:
            # print(df_missing) 
        
            # add a filter to save missing because of empty geometry but not empty area
            # look at what is left
            emptygeodf = df_missing[df_missing['Latitude'].isna()]
            emptygeodf = emptygeodf.dropna()
            # print(emptygeodf)        
            nonemptygeodf = df_missing[~df_missing['Latitude'].isna()]
            for row in nonemptygeodf.index:
                print(nonemptygeodf.loc[row].to_frame().T.to_string(index=False))
        

    

KeyError: "['Fuel'] not in index"

In [ ]:
# FIND 8 missing

lastegtfile = '/Users/gem-tah/Downloads/europe_2025-02-25.geojson'
currentegtfile = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-05-02.geojson'

In [ ]:
# Read the parquet files into GeoDataFrames
lastegt = gpd.read_file(lastegtfile)
currentegt = gpd.read_file(currentegtfile) # filter down to europe only

for df in [lastegt, currentegt]:
    print(df.columns)

no_geo = lastegt[lastegt['geometry'].is_empty]
no_geo2 = currentegt[currentegt['geometry'].is_empty]
no_geo_all = pd.concat([no_geo, no_geo2])
print(len(no_geo_all))

# Ensure both GeoDataFrames have a common column to compare (e.g., 'url')
common_column = 'url'

# Find rows in currentegt that are not in lastegt
missing_in_lastegt = currentegt[~currentegt[common_column].isin(lastegt[common_column])]

# Find rows in lastegt that are not in currentegt
missing_in_currentegt = lastegt[~lastegt[common_column].isin(currentegt[common_column])]

# Display the differences
print("Rows in currentegt but not in lastegt:")
print(missing_in_lastegt)

print("\nRows in lastegt but not in currentegt:")
print(missing_in_currentegt)


NameError: name 'lastegtfile' is not defined

In [ ]:
ggit_egt = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/latest/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02_DATA_TEAM_COPY.parquet'

ggit_gaspipes = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/latest/GEM-GGIT-Gas-Pipelines-2024-12_DATA_TEAM_COPY.parquet'


In [ ]:
# Read the parquet files into GeoDataFrames
egt_gdf = pd.read_parquet(ggit_egt)
gaspipes_gdf = pd.read_parquet(ggit_gaspipes) # filter down to europe only

egt_gdf = egt_gdf[['Wiki', 'geometry']]
gaspipes_gdf = gaspipes_gdf[['Wiki', 'geometry']]

for df in [egt_gdf, gaspipes_gdf]:
    print(df.columns)
    
#     df['geometry'] = df['geometry'].apply(lambda geom: wkt.loads(geom) if geom else None)

#     gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Ensure both GeoDataFrames have a common column to compare (e.g., 'url')
common_column = 'Wiki'

# Find rows in egt_gdf that are not in gaspipes_gdf
missing_in_gaspipes = egt_gdf[~egt_gdf[common_column].isin(gaspipes_gdf[common_column])]

# Find rows in gaspipes_gdf that are not in egt_gdf
missing_in_egt = gaspipes_gdf[~gaspipes_gdf[common_column].isin(egt_gdf[common_column])]

# Display the differences
print("Rows in EGT but not in GasPipes:")
print(missing_in_gaspipes)

print("\nRows in GasPipes but not in EGT:")
print(missing_in_egt)

# check if the 5 missing from new egt are in the egt file

missing = [
    "https://www.gem.wiki/Leningrad-Vyborg-State_Border_Gas_Pipeline",
    "https://www.gem.wiki/Gryazovets-Vyborg_Gas_Pipeline",
    "https://www.gem.wiki/Novopskov-Aksai-Mozdok_Pipeline",
    "https://www.gem.wiki/Stavropol_Moscow_Pipeline",
    "https://www.gem.wiki/Azerbaijan-Georgia-Romania_Interconnector_(AGRI)"
]

missing_not_in_egt = [link for link in missing if link not in egt_gdf[common_column].values]

print("Links in 'missing' that are not in egt_gdf:")
print(missing_not_in_egt)


#     

# read the parquet files, see what pipelines are missing between egt and gaspipes

Index(['Wiki', 'geometry'], dtype='object')
Index(['Wiki', 'geometry'], dtype='object')
Rows in EGT but not in GasPipes:
                                                   Wiki  \
27    https://www.gem.wiki/Bosnia_and_Herzegovina%E2...   
30    https://www.gem.wiki/Ionian_Adriatic_Gas_Pipeline   
79       https://www.gem.wiki/H2ercules_Pipeline_System   
81       https://www.gem.wiki/H2ercules_Pipeline_System   
82       https://www.gem.wiki/H2ercules_Pipeline_System   
...                                                 ...   
1267       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1268       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1269       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1270       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1271   https://www.gem.wiki/Pomeranian_Hydrogen_Cluster   

                                               geometry  
27    MULTILINESTRING ((16.3718 43.573, 16.5838 43.5...  
30    LINESTRING (19.559838 40.726419, 19.542955 40...

In [ ]:
# EU gas pipelines discrepancy
# previous 978
# current 966 
# Missing 12
# now missing 7 after patchingin 5 from last europe map file
old_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-02-25.geojson'
new_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-05-02.geojson'

# find which units are missing in new file

# Load the old and new files as GeoDataFrames
old_gdf = gpd.read_file(old_file)
new_gdf = gpd.read_file(new_file)

# Ensure the data contains a column that uniquely identifies each pipeline (e.g., 'Pipeline ID' or 'Name')
# Replace 'Pipeline ID' with the actual column name that uniquely identifies pipelines

# old_pipelines = old_gdf[old_gdf['tracker-display']=='gas pipeline']  # Adjust columns as needed
# new_pipelines = new_gdf[new_gdf['tracker-display']=='gas pipeline']  # Adjust columns as needed

old_gdf = old_gdf[~old_gdf['geometry'].is_empty]
new_gdf = new_gdf[~new_gdf['geometry'].is_empty]
# old_pipelines = old_pipelines[['url', 'name']]  # Adjust columns as needed
# new_pipelines = new_pipelines[['url', 'name']]  # Adjust columns as needed

# Find pipelines in the old file that are missing in the new file
missing_pipelines = old_gdf[~old_gdf['url'].isin(new_gdf['url'])]

# Display the missing pipelines
print(missing_pipelines)
# Save the missing pipelines to a GeoJSON file
missing_pipelines.to_file(f"missing_pipelines_egt_2025-05-02.geojson", driver="GeoJSON")

         id                                                url  \
1262  P5590  https://www.gem.wiki/Leningrad-Vyborg-State_Bo...   
1312  P0746  https://www.gem.wiki/Gryazovets-Vyborg_Gas_Pip...   
1417  P1471  https://www.gem.wiki/Novopskov-Aksai-Mozdok_Pi...   
1489  P0793     https://www.gem.wiki/Stavropol_Moscow_Pipeline   
1503  P0735  https://www.gem.wiki/Azerbaijan-Georgia-Romani...   

                                                   name unit-name     status  \
1262         Leningrad-Vyborg-State Border Gas Pipeline        II  operating   
1312                     Gryazovets-Vyborg Gas Pipeline         I  operating   
1417                    Novopskov-Aksai-Mozdok Pipeline            operating   
1489                      Stavropol-Moscow Gas Pipeline              retired   
1503  Azerbaijan-Georgia-Romania Interconnector Gas ...             proposed   

                     areas other-name fuel  \
1262      Estonia; Russia;             Gas   
1312               Russia;    

In [ ]:
# fix current egt data
missing = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/missing_pipelines_egt_2025-05-02.geojson'
current = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-05-05.geojson'
path_eu = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/'

missing_gdf = gpd.read_file(missing)
current_gdf = gpd.read_file(current)

# Concatenate the GeoDataFrames
fixed = gpd.GeoDataFrame(pd.concat([missing_gdf, current_gdf], ignore_index=True))


# Save the resulting GeoDataFrame if needed
fixed.to_file(f"{path_eu}europe_i5_2025-05-05.geojson", driver="GeoJSON")



In [ ]:
# Extract all unique countries from the 'areas' column, split by ';', and alphabetize them
countries = sorted(set(country.strip() for area in fixed['areas'] for country in area.split(';') if country.strip()))
countries

['Albania',
 'Algeria',
 'Austria',
 'Azerbaijan',
 'Belarus',
 'Belgium',
 'Benin',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Egypt',
 'Estonia',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Guinea',
 'Guinea-Bissau',
 'Hungary',
 'Iran',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Jordan',
 'Kazakhstan',
 'Kosovo',
 'Latvia',
 'Liberia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Mauritania',
 'Moldova',
 'Montenegro',
 'Morocco',
 'Netherlands',
 'Nigeria',
 'North Macedonia',
 'Norway',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russia',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'Sierra Leone',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Syria',
 'The Gambia',
 'Togo',
 'Tunisia',
 'Türkiye',
 'Ukraine',
 'United Kingdom']

In [ ]:
# list of statuses for closed mines
gcmt = 'coal-mine/compilation_output/gcmt_2025-05-05.geojson'
gcmtdf = gpd.read_file(gcmt)


In [ ]:
statuses_all = set(gcmtdf['status'].to_list())
statuses_all

{'cancelled', 'mothballed', 'not-found', 'operating', 'proposed', 'shelved'}

In [ ]:
# Define the column headers as lists
plants_columns = [
    "Wiki URL", "Country/Area", "Plant name", "Plant Name in Local Language / Script", "Other Name(s)", 
    "Unit name", "Fuel", "Capacity (MW)", "Status", "Turbine/Engine Technology", 
    "Equipment Manufacturer/Model", "CHP", "Hydrogen capable?", "CCS attachment?", 
    "Conversion/replacement?", "Conversion from/replacement of (fuel)", 
    "Conversion from/replacement of (GEM unit ID)", "Conversion to (fuel)", 
    "Conversion to (GEM unit ID)", "Start year", "Retired year", "Planned retire", 
    "Operator(s)", "Owner(s)", "Parent(s)", "Latitude", "Longitude", "Location accuracy", 
    "City", "Local area (taluk, county)", "Major area (prefecture, district)", 
    "State/Province", "Subregion", "Region", "Other IDs (location)", "Other IDs (unit)", 
    "Captive industry use", "Captive industry type", "Captive non-industry use", 
    "GEM location ID", "GEM unit ID"
]

plants_hy_columns = [
    "In Oil & Gas Plants tab?", "Wiki URL", "Country/Area", "Plant name", 
    "Plant Name in Local Language / Script", "Other Name(s)", "Unit name", "Fuel", 
    "Capacity (MW)", "Status", "Turbine/Engine Technology", "Equipment Manufacturer/Model", 
    "CHP", "Hydrogen capable?", "CCS attachment?", "Conversion/replacement?", 
    "Conversion from/replacement of (fuel)", "Conversion from/replacement of (GEM unit ID)", 
    "Conversion to (fuel)", "Conversion to (GEM unit ID)", "Start year", "Retired year", 
    "Planned retire", "Operator(s)", "Owner(s)", "Parent(s)", "Latitude", "Longitude", 
    "Location accuracy", "City", "Local area (taluk, county)", "Major area (prefecture, district)", 
    "State/Province", "Subregion", "Region", "Other IDs (location)", "Other IDs (unit)", 
    "Captive industry use", "Captive industry type", "Captive non-industry use", 
    "GEM location ID", "GEM unit ID", "H2 usage proposed %", "Color of H2 proposed?", 
    "H2 ready turbine (%)?", "MOU for H2 supply?", "Contract for H2 supply?", 
    "Financing for supply of H2?", "Co-located with electrolyzer/H2 production facility?", 
    "What % of H2 blending currently?"
]

# Find the differences between the two lists
not_in_plants_hy = list(set(plants_columns) - set(plants_hy_columns))
not_in_plants = list(set(plants_hy_columns) - set(plants_columns))

# Combine the results into a single list
non_overlapping_columns = {
    "Not in plants_hy_columns": not_in_plants_hy,
    "Not in plants_columns": not_in_plants
}

non_overlapping_columns



{'Not in plants_hy_columns': [],
 'Not in plants_columns': ['MOU for H2 supply?',
  'In Oil & Gas Plants tab?',
  'What % of H2 blending currently?',
  'Contract for H2 supply?',
  'H2 usage proposed %',
  'Financing for supply of H2?',
  'Co-located with electrolyzer/H2 production facility?',
  'Color of H2 proposed?',
  'H2 ready turbine (%)?']}

In [ ]:
# file = 'local_pkl/about_df_dict_by_map_2025-02-07.pkl'
filegoit =  'GOIT/compilation_output/goit_2025-04-08.geojson'
# file_goget_json = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/goget/compilation_output/Oil & Gas Extraction-map-file-2025-02-20.csv'

In [ ]:
# https://drive.google.com/file/d/1wy53Z56YeIad4Tod46tGcOmduxBPLZ7r/view?usp=drive_link can we access via this link?

ggit_lng_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson'
ggit_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-Gas-Pipelines-2024-12 DATA TEAM COPY.geojson' #'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-Gas-Pipelines-2023-12 copy.geojson'

ggit_lng_eu_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-EGT-Terminals-2025-02 DATA TEAM COPY.geojson'
ggit_eu_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02 DATA TEAM COPY.geojson'

list_of_geojsons = [ggit_lng_geojson, ggit_geojson, ggit_lng_eu_geojson, ggit_eu_geojson]

for filepath in list_of_geojsons:
    gdf = gpd.read_file(filepath)
    print(gdf.shape)
    # newfilepath = '/'.join(filepath.split('/')[:-1])
    newfilepath = filepath.replace('.geojson', '.parquet').replace(' ', '_')
    print(f'{newfilepath}')
    gdf.to_parquet(path = f'{newfilepath}')
    # save_file_to_s3.saves3(f'{filepath}') # also save original
    save_file_to_s3.saves3(newfilepath)

(1295, 59)
/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson


CalledProcessError: Command 'export BUCKETEER_BUCKET_NAME=publicgemdata && aws s3 cp /Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson s3://$BUCKETEER_BUCKET_NAME/latest/ --endpoint-url https://nyc3.digitaloceanspaces.com --acl public-read' returned non-zero exit status 252.

In [ ]:
# save most recent download to s3


In [ ]:
# list_of_links = ['https://docs.google.com/spreadsheets/d/1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0/', 'https://drive.google.com/file/d/1X6tpYmgDKUWZzSYGlz0LcCeE-DqGeQcp/view?usp=drive_link', 
#                  'https://docs.google.com/spreadsheets/d/1aZiKzbHfFXLSEFCeKwstvL1-DV98ErXDhyCx00abRgQ/', 'https://docs.google.com/spreadsheets/d/1dqawuLhgFzYI9GWRU9bwsDY1SQHjzxifZb7u1T8OXqA/',
#                  'https://docs.google.com/spreadsheets/d/1zUyj7t4FO0M9fc82wpG93fZmisGar0Syq1l5ZGyyxEM/', 'https://docs.google.com/spreadsheets/d/1TxoQ17GUDvRbQ82r90eexbRNHfPfi-50wm9lBih2eYc/',
#                  'https://docs.google.com/spreadsheets/d/1hB2NuXMMqCJD94muoe6-A9BCi4v-7hquScej44_QWas/', 'https://docs.google.com/spreadsheets/d/1crtLnGzOS6wLYweIb84gLQpMV4A49qD5fty6WBTdLt0/',
#                  'https://docs.google.com/spreadsheets/d/1jUbsItBq_CVXy0dUxCP7APHfstDL85BaykCXYURagX4/', 'https://docs.google.com/spreadsheets/d/1bUTPRTmIqu7URVECm66NGr_DF7v32UUOzBaBf1dNawk/',
#                  'https://docs.google.com/spreadsheets/d/1x-IxdiqMJ_v-M_45ASFdg55Gl3Hsx3vb-7wjt8qCq5k/', 'https://docs.google.com/spreadsheets/d/1cy0A7k0GoB2b_b2wesObcAvts0GbXa7go7FVE_bHpBc/',
#                  'https://docs.google.com/spreadsheets/d/1XxAvcobKZ9XRKvHaSHDb4JbOstqhvrVWkU5Wu8JKhgE/', 'https://docs.google.com/spreadsheets/d/1gOOwjN551r6TllpBpfxtwNqF1yfbLbXdCFwACvKYeI4/',
#                  'https://docs.google.com/spreadsheets/d/10Clsb8auutXy4iBm6c7HywZjSbNKUiho9NHzZmC2-Bs/', 'https://docs.google.com/spreadsheets/d/1cqLe0xOx7FLpfaGZhtVnV-bhuKgaczHPtXzCRIgy_RQ/',
#                  'https://docs.google.com/spreadsheets/d/1zCcO9LwVmiQZBOGqlwKGYgZ5lAnBKJ91hjwWcbUw5nQ/', 'https://docs.google.com/spreadsheets/d/1zCcO9LwVmiQZBOGqlwKGYgZ5lAnBKJ91hjwWcbUw5nQ/',
#                  'https://docs.google.com/spreadsheets/d/16MoNKcz68GVwv6ztZ6p0vSNlL6EmPd633DZ7V0xG_Ho/']


# for link in list_of_links:
    
# save_file_to_s3.googlelinktoparquet(link)

Most recent file found: /Users/gem-tah/Downloads/Global Coal Project Finance Tracker_April 2025_Global Energy Monitor DATA TEAM COPY.xlsx
most_recent_file_name: Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY
File saved as Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY_1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0_2025-04-14.parquet
upload: ./Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY_1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0_2025-04-14.parquet to s3://publicgemdata/latest/Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY_1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0_2025-04-14.parquet

Most recent file found: /Users/gem-tah/Downloads/Global Coal Project Finance Tracker_April 2025_Global Energy Monitor DATA TEAM COPY.xlsx
most_recent_file_name: Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY
File saved a

KeyboardInterrupt: 

In [ ]:
df = gpd.read_file(filegoit)


In [ ]:
# df.to_parquet(f"goit-map-2025-03.parquet", index=False)  # partition_cols=["country/area"],
# file = 'goit-map-2025-03.parquet'
file = filegoit

do_command_s3 = (
            f'export BUCKETEER_BUCKET_NAME=publicgemdata && '
            f'aws s3 cp {file} s3://$BUCKETEER_BUCKET_NAME/latest/ '
            f'--endpoint-url https://nyc3.digitaloceanspaces.com --acl public-read')

        # Execute the terminal command to pull down file from digital ocean
process = subprocess.run(do_command_s3, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print(process.stdout.decode('utf-8'))
if process.stderr:
    print(process.stderr.decode('utf-8'))


upload: GOIT/compilation_output/goit_2025-04-08.geojson to s3://publicgemdata/latest/goit_2025-04-08.geojson



In [ ]:
set(df['status-legend'].to_list())

{'cancelled',
 'construction-plus',
 'mothballed-plus',
 'operating',
 'proposed-plus',
 'retired-plus',
 'shelved'}

In [ ]:
# df = pickle.load(file)
# df = gpd.read_file(file_goget_json)

In [ ]:

# country_list = set(df['country-list'].to_list())
# print(country_list)

countries = set(df['country/area'].apply(lambda x: x.split('-')).apply(lambda x: '; '.join(x)).to_list())
print(sorted(countries))

# missing = country_list - countries


# print(missing)

In [ ]:
current_countries = {
  "Africa": [
    "Algeria",
    "Angola",
    "Angola-Republic of the Congo",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "Côte d'Ivoire",
    "Djibouti",
    "DR Congo",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Republic of the Congo",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Senegal-Mauritania",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ],
  "Americas": [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bermuda",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Canada",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Greenland",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Puerto Rico",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Pierre and Miquelon",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "United States",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)",
    "Virgin Islands (U.S.)"
  ],
  "Asia": [
    "Afghanistan",
    "Armenia",
    "Azerbaijan",
    "Azerbaijan-Turkmenistan",
    "Bahrain",
    "Bangladesh",
    "Bhutan",
    "Brunei",
    "Cambodia",
    "China",
    "China-Japan",
    "Cyprus",
    "Georgia",
    "Hong Kong",
    "India",
    "Indonesia",
    "Iran",
    "Iran-Iraq",
    "Iraq",
    "Israel",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kuwait",
    "Kuwait-Saudi Arabia",
    "Kuwait-Saudi Arabia-Iran",
    "Kyrgyzstan",
    "Laos",
    "Lebanon",
    "Macao",
    "Malaysia",
    "Maldives",
    "Mongolia",
    "Myanmar",
    "Nepal",
    "North Korea",
    "Oman",
    "Pakistan",
    "Palestine",
    "Philippines",
    "Qatar",
    "Russia-Kazakhstan",
    "Saudi Arabia",
    "Saudi Arabia-Bahrain",
    "Saudi Arabia-Iran",
    "Singapore",
    "South China Sea",
    "South Korea",
    "Sri Lanka",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Thailand",
    "Thailand-Malaysia",
    "Timor-Leste",
    "Türkiye",
    "Turkmenistan",
    "United Arab Emirates",
    "United Arab Emirates-Iran",
    "Uzbekistan",
    "Vietnam",
    "Vietnam-Malaysia",
    "Yemen"
  ],
  "Europe": [
    "Åland Islands",
    "Albania",
    "Andorra",
    "Austria",
    "Belarus",
    "Belgium",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Faroe Islands",
    "Finland",
    "France",
    "Germany",
    "Gibraltar",
    "Greece",
    "Guernsey",
    "Holy See",
    "Hungary",
    "Iceland",
    "Ireland",
    "Isle of Man",
    "Italy",
    "Jersey",
    "Kosovo",
    "Latvia",
    "Liechtenstein",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Moldova",
    "Monaco",
    "Montenegro",
    "Netherlands",
    "North Macedonia",
    "Norway",
    "Poland",
    "Portugal",
    "Romania",
    "Russia",
    "Russia-Kazakhstan",
    "San Marino",
    "Serbia",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Svalbard and Jan Mayen",
    "Sweden",
    "Switzerland",
    "Ukraine",
    "United Kingdom"
  ],
  "Oceania": [
    "American Samoa",
    "Australia",
    "Christmas Island",
    "Cocos (Keeling) Islands",
    "Cook Islands",
    "Fiji",
    "French Polynesia",
    "Guam",
    "Heard Island and McDonald Islands",
    "Kiribati",
    "Marshall Islands",
    "Micronesia",
    "Nauru",
    "New Caledonia",
    "New Zealand",
    "Niue",
    "Norfolk Island",
    "Northern Mariana Islands",
    "Palau",
    "Papua New Guinea",
    "Pitcairn",
    "Samoa",
    "Solomon Islands",
    "Timor Gap",
    "Tokelau",
    "Tonga",
    "Tuvalu",
    "United States Minor Outlying Islands",
    "Vanuatu",
    "Venezuela-Trinidad and Tobago",
    "Wallis and Futuna"
  ]
}


current_countries_list = []

for v in current_countries.values():
    current_countries_list.extend(v)

print(current_countries_list)

print(current_countries_list)

In [ ]:
countries_list = list(countries)
countries_list = [c.replace('; ', '-') for c in countries_list]

missing_countries_areas = set(countries_list) - set(current_countries_list)

In [ ]:
missing_countries_areas

In [ ]:
# add missing_countries_areas to current_countries 


compare all regional outputs and source data

In [ ]:
folder_path_pkl = f'{tracker_folder_path}local_pkl'


for folder in ['LATAM', 'africa-energy', 'asia']:
    
    file_path = f'{folder}/config.js'
    if folder == 'africa-energy':
        folder = 'africa'
        
    with open(file_path, 'r') as file:
        jscontent = file.read()
        # print(type(jscontent)) # string
        # print(jscontent)
        # make df from json file 
        if 'geojson:' in jscontent:
            second_part = jscontent.split('json: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]
            folder_full = tracker_folder_path + str(folder)


            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')
            current_file_line = str(current_file_line)
            print(current_file_line)
            current_file_line_csv = current_file_line.replace('geojson', 'csv')

            full_path = os.path.join(folder_full,current_file_line)

            # df = pd.read_json(full_path)
            # change to csv version that is / should be saved next to it b/c easier to read
            full_path_csv = os.path.join(folder_full,current_file_line_csv)
            try:
                df = pd.read_csv(full_path_csv)
            except FileNotFoundError:
                current_file_line_xlsx = current_file_line.replace('json', 'xlsx')
                full_path_xlsx = os.path.join(folder_full,current_file_line_xlsx)
                print(full_path_xlsx)
                try: 
                    df = pd.read_excel(full_path_xlsx)
                    
                except FileNotFoundError:
                    print(f'Check the folder {folder_full} seemingly no alternatives to json tried csv and xlsx')
            finally:
                # print(df.head())
                print(f'Length df: {len(df)}')

                
        
        elif 'json' in jscontent:
            second_part = jscontent.split('json: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]

            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')

            folder_full = tracker_folder_path + str(folder)

            current_file_line = str(current_file_line)
            print(current_file_line)
            current_file_line_csv = current_file_line.replace('geojson', 'csv')

            full_path = os.path.join(folder_full,current_file_line)

            # df = pd.read_json(full_path)
            # change to csv version that is / should be saved next to it b/c easier to read
            full_path_csv = os.path.join(folder_full,current_file_line_csv)
            try:
                df = pd.read_csv(full_path_csv)
            except FileNotFoundError:
                current_file_line_xlsx = current_file_line.replace('geojson', 'xlsx')
                full_path_xlsx = os.path.join(folder_full,current_file_line_xlsx)
                print(full_path_xlsx)
                try: 
                    df = pd.read_excel(full_path_xlsx)
                except FileNotFoundError:
                    print(f'Check the folder {folder_full} seemingly no alternatives to json tried csv and xlsx')
            finally:
                # print(df.head())
                # print(len(df))
                print(f'Length df: {len(df)}')

            
        elif 'csv:' in jscontent:
            second_part = jscontent.split('csv: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]

            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')


            folder_full = tracker_folder_path + str(folder)
            full_path = os.path.join(folder_full,current_file_line)
            df = pd.read_csv(full_path)
            print(f'Length df: len(df)')

            
        else:
            print(jscontent)

        print(date_pkl)
        # filter for tracker to update 
        print(folder)
        print(len(df))
        df_filt = df[df["tracker-acro"]=='GOGET']
        print(len(df_filt))

            
    

totals:
AGT: Map - 474, table - 479 (potentially Thailand-Malaysia, China-Japan, Vietnam-Malaysia missing)
LATAM - map - 1195, table - 1200 (missing- Venezuela-Trinidad and Tobago?)
AET:Map - 665, table 667 (Senegal-Mauritania, Angola-Republic of the Congo)

In [ ]:
# pull in source
# filter for region and fuel
folder_path_pkl = f'{tracker_folder_path}local_pkl'
file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'

full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
df_pkl = pd.read_pickle(full_path_pkl)
f'Length of source after being filtered: {len(df_pkl)}'


Check the summary colab to see how he filters region and fuel, must be that. 
And pull in europe to this slew of functions so have those pkl files too

From scotts summary code 
Regional Tracker Summary Tables
https://colab.research.google.com/drive/1c1Y7MHV6f8lEAeb8l0nw1B_d1ETmwVRb#scrollTo=e4yXRRIkch6v 

In [ ]:
# AET covers "The Africa Energy Tracker covers the subregions of Northern Africa and Sub-Saharan Africa" https://globalenergymonitor.org/projects/africa-energy-tracker/methodology/
# Define African Sites
african_countries = [
    "Algeria", "Angola", "Angola-Republic of the Congo", "Benin", "Botswana",
    "British Indian Ocean Territory", "Burkina Faso", "Burundi", "Cabo Verde",
    "Cameroon", "Central African Republic", "Chad", "Comoros", "Côte d'Ivoire",
    "Djibouti", "DR Congo", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini",
    "Ethiopia", "French Southern Territories", "Gabon", "Ghana", "Guinea",
    "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar",
    "Malawi", "Mali", "Mauritania", "Mauritius", "Mayotte", "Morocco",
    "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo",
    "Réunion", "Rwanda", "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe", "Senegal", "Senegal-Mauritania", "Seychelles",
    "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan",
    "Tanzania", "The Gambia", "Togo", "Tunisia", "Uganda", "Western Sahara",
    "Zambia", "Zimbabwe"
]


# Define Asian Countries https://globalenergymonitor.org/projects/asia-gas-tracker/methodology/
asia_countries = [
    # South Asia
    "Afghanistan", "Bangladesh", "Bhutan", "India", "Iran",
    "Maldives", "Nepal", "Pakistan", "Sri Lanka",

    # Southeast Asia
    "Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia",
    "Myanmar", "Philippines", "Singapore", "Thailand",
    "Timor-Leste", "Vietnam", "Thailand-Malaysia",  "Vietnam-Malaysia",

    # East Asia
    "China", "China-Japan", "Hong Kong", "Japan", "Macao",
    "Mongolia", "North Korea", "South Korea", "Taiwan",

    # Multinational or Maritime Areas
    "South China Sea"
]

# Define European Countries and Territories https://globalenergymonitor.org/projects/europe-gas-tracker/methodology/
europe_countries = [
    # Northern Europe
    "Åland Islands", "Denmark", "Estonia", "Faroe Islands", "Finland",
    "Iceland", "Ireland", "Isle of Man", "Latvia", "Lithuania",
    "Norway", "Sweden", "United Kingdom", "Guernsey", "Jersey", "Svalbard and Jan Mayen",

    # Western Europe
    "Austria", "Belgium", "France", "Germany", "Liechtenstein",
    "Luxembourg", "Monaco", "Netherlands", "Switzerland",

    # Eastern Europe
    "Belarus", "Czech Republic", "Hungary", "Moldova", "Poland",
    "Romania", "Slovakia", "Ukraine",

    # Southern Europe
    "Albania", "Andorra", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Cyprus", "Gibraltar", "Greece", "Italy", "Kosovo", "Malta",
    "Montenegro", "North Macedonia", "Portugal", "San Marino",
    "Serbia", "Slovenia", "Spain", "Türkiye",

    # Special Cases
    "Israel"  # list in the methodology
]

# Define Latin American and Caribbean Countries and Territories - see "Data request for the new year"
latin_america_caribbean = [
    # Caribbean
    "Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados",
    "Belize", "Cayman Islands", "Cuba", "Curaçao", "Dominica",
    "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica",
    "Martinique", "Montserrat",  "Saint Barthélemy",
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin (French part)",
    "Saint Vincent and the Grenadines", "Sint Maarten (Dutch part)",
    "Trinidad and Tobago", "Turks and Caicos Islands", "Virgin Islands (British)",
   # "Virgin Islands (U.S.)", "Puerto Rico", -  gregor excludes

    # Central America
    "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico",
    "Nicaragua", "Panama",

    # South America
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador",
    "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname",
    "Uruguay", "Venezuela",

    # Special Cases
    "Bonaire, Sint Eustatius, and Saba", "Bouvet Island",
    "Falkland Islands", "South Georgia and the South Sandwich Islands",
    "Venezuela-Trinidad and Tobago"
]

In [ ]:
# map countries 


africa_countries_map = [
    "Algeria",
    "Angola",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "DR Congo",
    "Republic of the Congo",
    "Côte d'Ivoire",
    "Djibouti",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ]


asia_countries_map = [ 
        "China",
        "Hong Kong",
        "Japan",
        "Macao",
        "Mongolia",
        "North Korea",
        "South Korea",
        "Taiwan",
        "Brunei",
        "Cambodia",
        "Indonesia",
        "Laos",
        "Malaysia",
        "Myanmar",
        "Philippines",
        "Singapore",
        "Thailand",
        "Timor-Leste",
        "Vietnam",
        "Afghanistan",
        "Iran",
        "Bangladesh",
        "Bhutan",
        "India",
        "Maldives",
        "Nepal",
        "Pakistan",
        "Sri Lanka"]

# european_union_countries = [
#     'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
#     'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 
#     'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 
#     'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 
#     'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 
#     'Spain', 'Sweden',
# ]
# other_europe_countries = [
#     'Albania', 'Andorra', 'Belarus', 'Bosnia and Herzegovina', 'Holy See', 'Iceland',
#     'Liechtenstein', 'Moldova', 'Monaco', 'Montenegro', 'North Macedonia', 
#     'Norway', 'San Marino', 'Serbia', 'Switzerland', 'Türkiye', 'Ukraine', 
#     'United Kingdom', 'Israel',
# ]

europe_countries_map = [
    'Åland Islands', 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 
    'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czech Republic', 
    'Denmark', 'Estonia', 'Faroe Islands', 'Finland', 'France', 'Germany', 
    'Gibraltar', 'Greece', 'Guernsey', 'Holy See', 'Hungary', 'Iceland', 
    'Ireland', 'Isle of Man', 'Italy', 'Jersey', 'Kosovo', 'Latvia', 
    'Liechtenstein', 'Lithuania', 'Luxembourg', 'North Macedonia', 'Malta', 
    'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 
    'Portugal', 'Romania', 'Israel', 'San Marino', 'Serbia', 'Slovakia', 
    'Slovenia', 'Spain', 'Svalbard and Jan Mayen', 'Sweden', 'Switzerland', 
    'Ukraine', 'United Kingdom', 'Cyprus', 'Türkiye'
]



latam_countries_map = [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)"]

In [ ]:
#compare differenc with subtract a set

missing_eu = set(europe_countries) - set(europe_countries_map) 
# missing_eu = set(europe_countries_map) - set(europe_countries) 

print(missing_eu)

missing_asia = set(asia_countries) - set(asia_countries_map)
# missing_asia = set(asia_countries_map) - set(asia_countries)

print(missing_asia)

missing_africa = set(african_countries) - set(africa_countries_map)
# missing_africa = set(africa_countries_map) - set(african_countries)

print(missing_africa)

missing_latam = set(latin_america_caribbean) - set(latam_countries_map)
# missing_latam = set(latam_countries_map) - set(latin_america_caribbean)

print(missing_latam)



In [ ]:
# map 949 goget eu
# scott 964.0 goget eu

GOGET architecture exploring

In [ ]:
# 7k projects goget
goget_file = '/Users/gem-tah/Downloads/GOGET oil & gas extraction sites (main data set) - Main data.csv'

goget_df = pd.read_csv(goget_file)
print(set(goget_df['Project or complex'].to_list()))

# print(goget_df['Unit type'])
print(goget_df.columns)



{nan, '11416/Cold Lake', '10587DD/Athabasca', 'Carabobo-3', 'La Rona', 'Upper Wolfcamp East', 'Doe', '12689A/Athabasca', '8002F/Cold Lake', 'Girsan-Chulkuvar', 'Diatomite', '12229A/Cold Lake', '9349E/Cold Lake', 'Parkland', 'Penn Upper', '9637C/Cold Lake', 'Gannet', '12680A/Athabasca', 'Other Areas', 'Eastern Fields Oil and Gas Complex', 'MLNG Tiga', 'Brazion', 'Kenli 6-1', '10616/Peace River Area 2', '12316/Athabasca', '12795/Cold Lake', 'Xijiang 30-2', 'Sapphire', 'Gina Krog', 'Brush Creek', 'Peace River Area 2', 'Stevens', '13009/Peace River Area 2', '11480B/Athabasca', 'Maari', 'Wilcox 10400', '11256I/Cold Lake', 'Gela', 'Jedney', 'Surgil', 'North East Bab', 'Carneros', 'Coatzacoalcos-Marino; Area 1', 'Devonian', 'Buzzard Creek', 'Kurdistan', '11320F/Peace River Area 2', 'Pettit, Lower Gas', 'Banyu Urip', 'Jujo-Tecominoacán', 'Cv', 'Cold Lake Area - Scheme 8908T Oil Sands', 'Narwhal', 'Julimar-Brunello', 'Liuhua 16-2', 'Serrette', '11729A/Cold Lake', '10070J/Peace River Area 1', '1

In [ ]:
projects = goget_df[goget_df['Unit type']=='project']
not_projects = goget_df[goget_df['Unit type']!='project']
print(len(projects))
print(len(not_projects))

not_in_project = goget_df[goget_df['Project or complex'].isna()]
print(len(not_in_project))

print(len(goget_df))

# are those that complexes that have no data but have fields associated with them get published in the official data download

# find the rows that are connected to a project,

         
# how many units are in a project 

626
14784
11094
15410


In [ ]:
# we want a list of all the goget units that do not agree with its project
#  Unit type = field, project, complex 
# Project or complex = name of corresponding project 

In [ ]:
hier_top = ['project', 'complex'] # block
hier_middle = ['field', 'area', 'asset', 'pool', 'phase']
hier_middle_new = ['field', 'area', 'asset', 'pool', 'phase', 'sub-basin', 'concession', 'basin']
# hier_low = [''] # 'sub-basin, concession, basin

cols_hier_top = ['Owner 1p', 'Owner 1f', 'Owner 1 %p', 'Owner 1 %f', 'Owner 2p', 'Owner 2f', 'Owner 2 %p', 'Owner 2 %f', 'Owner 3p', 'Owner 3f', 'Owner 3 %p', 'Owner 3 %f', 'Owner 4p', 'Owner 4f', 'Owner 4 %p', 'Owner 4 %f', 'Owner 5p', 'Owner 5f', 'Owner 5 %p', 'Owner 5 %f', 'Owner % sump', 'Owner % sumf'] # FID, start up, owner, onshore/offshore
cols_hier_middle = ['Latitudep', 'Latitudef', 'Longitudep', 'Longitudef', 'Location accuracyp', 'Location accuracyf', 'Operatorp', 'Operatorf'] # discover, operator, lat, long, basin, formation, block
cols_agree_needed = ['Country/Areap', 'Country/Areaf', 'Subnational unit (province, state)p', 'Subnational unit (province, state)f', 'Statusp', 'Statusf']
basic_cols = ['project ID (inferred)','project name (inferred)','Unit IDp','Unit IDf', 'Unit Namep','Unit Namef', 'Project or complexp', 'Project or complexf']

set(goget_df['Unit type'].to_list())
questions = '1.)what to do with 480 that arent clearly top or mid from that schema?'

In [ ]:
# field project groupings for existing hierarchy data reconciliation
# a field is in a grouping if its Unit type = field and its Project or complex is not blank
# a project is in a grouping if its Unit type = project or complex and in a pivot table it has fields
# find situations where field and project do not correspond, match on name 

# groupings = [] # tuples of project first, then list of fields
field_gr = goget_df.copy()
print(len(field_gr))
# field_gr = field_gr[field_gr['Unit type'].isin(hier_middle)]
field_gr = field_gr[field_gr['Unit type'].isin(hier_middle_new)]

field_gr = field_gr[field_gr['Project or complex'].notna()] # 14177 without this filter 3540 with it
field_gr['Project or complex'] = field_gr['Project or complex'].apply(lambda x:x.strip())
print(len(field_gr))
field_proj_names = field_gr['Project or complex'].to_list()
print(field_proj_names)

proj_gr = goget_df.copy()
print(len(proj_gr))
proj_gr = proj_gr[proj_gr['Unit type'].isin(hier_top)]
proj_gr['Unit Name'] = proj_gr['Unit Name'].apply(lambda x:x.strip())
print(len(proj_gr))
proj_names = proj_gr['Unit Name'].to_list()
print(proj_names)

no_overlap = set(field_proj_names) - set(proj_names) 
print(no_overlap)
print(len(goget_df)-len(proj_gr)-len(field_gr)) # 480 neither field or project 


15410
3543
['Matzen', 'Parque das Conchas', 'Iara, Atapu', 'Parque das Conchas', 'Iara, Atapu', 'Bacalhau', 'Bacalhau', 'Parque das Baleias', 'Iara', 'Búzios', 'Parque das Baleias', 'Parque das Baleias', 'Parque dos Gaviões', 'Parque dos Gaviões', 'Parque das Baleias', 'Lapa', 'Mero/Libra', 'Iara, Atapu', 'Parque das Conchas', 'Parque das Baleias', 'Sepia', 'Sepia', 'Iara', 'Iara', 'Aphrodite', 'Calypso', 'Glaucus', 'Lacq', 'Lacq', 'Düste', 'Düste', 'Emlichheim', 'Emlichheim', 'Kalle', 'Kalle', 'Ratzel', 'Ratzel', 'Ratzel', 'Rehden', 'Rehden', 'Rehden', 'Ringe', 'Ringe', 'Rütenbrock', 'Rütenbrock', 'Voelkersen', 'Wielen', 'Wielen', 'Wietingsmoor', 'Wietingsmoor', 'Algyo', 'Kinsale', 'Corrib', 'Inishkea', 'Kinsale', 'Kinsale', 'Kinsale', 'Karish-Tanin', 'Karish-Tanin', 'Leviathan', 'Tamar', 'Karish-Tanin', 'Val d’Agri', 'Val d’Agri', 'Val d’Agri', 'Gateway to the Ems', 'Grieg / Aasen', 'Valhall', 'Johan Castberg', 'Johan Sverdrup', 'B 8', 'BMB', 'Brońsko', 'Przemyśl', 'Midia Gas Develop

In [ ]:
# merge project rows with field rows 
print(cols_agree_needed + basic_cols)
groupings = proj_gr.merge(field_gr, left_on='Unit Name', right_on='Project or complex', how='inner', suffixes=('p', 'f'))
# rename Unit IDp to be project id for easier pivoting
# groupings.rename(columns={'Unit IDp': 'project ID (inferred)', 'Unit Namep': 'project name (inferred)'}, inplace=True)
groupings['project ID (inferred)'] = groupings['Unit IDp']
groupings['project name (inferred)'] = groupings['Unit Namep']
# len(groupings)
# print(groupings)


cols_rec_check = groupings[cols_agree_needed + basic_cols]
cols_rec_check_proj = groupings[cols_hier_top + basic_cols]
cols_rec_check_field = groupings[cols_hier_middle + basic_cols]

all_groupings = groupings[basic_cols + cols_agree_needed + cols_hier_top + cols_hier_middle]
# all_groupings.to_csv(f'all_groupings_goget 2025-03-10.csv')


['Country/Areap', 'Country/Areaf', 'Subnational unit (province, state)p', 'Subnational unit (province, state)f', 'Statusp', 'Statusf', 'project ID (inferred)', 'project name (inferred)', 'Unit IDp', 'Unit IDf', 'Unit Namep', 'Unit Namef', 'Project or complexp', 'Project or complexf']


In [ ]:

# groupings
# cols_rec_check
# cols_rec_check_proj
# cols_rec_check_field
# pivot 
# group on project ID and find inconsistencies
# pivot_cols = basic_cols.pop(2)
# del basic_cols[2] # project ID (inferred)
# pivot_cols = [basic_cols.pop(basic_cols.index('project ID (inferred)'))]
# pivot_cols = cols_agree_needed + cols_hier_top + cols_hier_middle
# all_groupings = all_groupings.groupby(['project ID (inferred)','project name (inferred)'])
# all_groupings = all_groupings.pivot(index='project ID (inferred)', columns = pivot_cols)
# print(all_groupings)

# a file showing side by side cols that do not match between project and field, but so that each field is lined up along project
# cols_hier_top = ['Owner 1', 'Owner 1 %', 'Owner 2', 'Owner 2 %', 'Owner 3', 'Owner 3 %', 'Owner 4', 'Owner 4 %', 'Owner 5', 'Owner 5 %', 'Owner % sum'] # FID, start up, owner, onshore/offshore
# cols_hier_middle = ['Latitude', 'Longitude', 'Location accuracy', 'Operator'] # discover, operator, lat, long, basin, formation, block
# cols_agree_needed = ['Country/Area', 'Subnational unit (province, state)', 'Status']
to_reconcile = {}
# loop through and see if values match, if they do not they get added to a tuple according to the project id
all_cols = cols_hier_top + cols_hier_middle + cols_agree_needed
# strip last character and de duplicate
all_cols = [col[:-1] for col in all_cols]
all_cols = list(set(all_cols))
all_groupings = all_groupings.fillna('')
for row in all_groupings.index:
    proj_id = all_groupings.loc[row, 'Unit IDp']
    proj_name = all_groupings.loc[row, 'Unit Namep']
    unit_name = all_groupings.loc[row, 'Unit Namef']
    unit_id = all_groupings.loc[row, 'Unit IDf']
    for col in all_cols:
        
        field_val = all_groupings.loc[row,f'{col}f']
        proj_val = all_groupings.loc[row,f'{col}p']
        if field_val != proj_val:
            if field_val == '' or proj_val == '':
                continue
            else:
                to_reconcile[(proj_id, proj_name, unit_id, unit_name, col)] = [field_val, proj_val]
                
to_reconcile_df = pd.DataFrame.from_dict(to_reconcile, orient='index')    
to_reconcile_df.to_excel(f'to_reconcile_goget_2025-03-11.xlsx', index=True)


In [ ]:


# print(f'Ask Scott: {questions}')

